In [209]:
import numpy as np
import os

In [210]:
def read_graph(file_path):
    G = {}
    nodes = []
    with open(file_path, 'r') as f:
        for line in f:
            from_node, to_node = map(int, line.strip().split())
            if from_node not in G:
                G[from_node] = []
            G[from_node].append(to_node)
            if from_node not in nodes:
                nodes.append(from_node)
            if to_node not in nodes:
                nodes.append(to_node)
    return G, nodes

In [211]:
def calculate_pagerank_and_sort(G, nodes, N, teleport_parameter):
    index = {}
    for i, node in enumerate(sorted(nodes)):
        index[node] = i

    S = np.zeros([N, N], dtype = np.float64)
    for from_node, to_nodes in G.items():
        for to_node in to_nodes:
            S[node_to_num[to_node], node_to_num[from_node]] = 1

    for j in range(N):
        sum_of_col = sum(S[:, j])
        if sum_of_col == 0:
            S[:, j] = 1 / N
        else:
            S[:, j] /= sum_of_col
    
    A = teleport_parameter * S + (1 - teleport_parameter) / N * np.ones([N, N], dtype = np.float64)
    
    P_n = np.ones(N, dtype = np.float64) / N
    P_n1 = np.zeros(N, dtype = np.float64)
    
    e = 100
    k = 0
    
    tol = 1 / (N * N)
    
    while e > tol:
        P_n1 = np.dot(A, P_n)
        e = P_n1 - P_n
        e = max(map(abs, e))
        P_n = P_n1
        k += 1
        print('iteration %s:' % str(k), P_n1)
    
    sorted_nodes = sorted(index.items(), key = lambda x: P_n[x[1]], reverse=True)
    
    sorted_results = []
    for node, index in sorted_nodes[:100]:
        sorted_results.append((node, P_n[index]))
    
    return sorted_results

In [212]:
def output_result(results, file_path):
    with open(file_path, 'w') as f:
        for node, score in results:
            f.write(f"{node} {score}\n")

In [213]:
if __name__ == '__main__':
    data_file_path = './Data.txt'
    output_file_path = './output/basic.txt'
    
    teleport_parameter = 0.85
    
    G, nodes= read_graph(data_file_path)
    N = len(nodes)
    sorted_results = calculate_pagerank_and_sort(G, nodes, N, teleport_parameter)
    output_result(sorted_results, output_file_path)

iteration 1: [2.06882149e-04 1.24164109e-04 1.50484011e-04 ... 1.22702895e-04
 1.07847828e-04 9.05925004e-05]
iteration 2: [2.74021221e-04 1.17440797e-04 1.37659859e-04 ... 1.16760210e-04
 1.02040058e-04 8.92870841e-05]
iteration 3: [3.27525325e-04 1.12904770e-04 1.33091481e-04 ... 1.11770189e-04
 9.78461627e-05 8.59158709e-05]
iteration 4: [3.70258577e-04 1.09456722e-04 1.28983245e-04 ... 1.08361125e-04
 9.49302953e-05 8.33600536e-05]
iteration 5: [4.04376542e-04 1.06724748e-04 1.25676856e-04 ... 1.05681386e-04
 9.26457156e-05 8.13847307e-05]
iteration 6: [4.31620523e-04 1.04547140e-04 1.23043989e-04 ... 1.03530048e-04
 9.08181136e-05 7.98160880e-05]
iteration 7: [4.53375468e-04 1.02808187e-04 1.20941629e-04 ... 1.01812817e-04
 8.93592971e-05 7.85639239e-05]
iteration 8: [4.70747402e-04 1.01419572e-04 1.19263102e-04 ... 1.00441590e-04
 8.81941758e-05 7.75639791e-05]
iteration 9: [4.84619374e-04 1.00310740e-04 1.17922792e-04 ... 9.93465551e-05
 8.72637752e-05 7.67654850e-05]
iteration 